In [1]:

import toml
import importlib
from torch.utils.data import DataLoader
import datetime
from PIL import Image
from utils import check_device
import numpy as np
import glob
import os
from inspect import isfunction
import torch
from Explanation.evaluation.tools import CausalMetric
device = check_device()
experiment_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
import pickle
adversarial_config = toml.load("Config/adversarial.toml")
explanation_config = toml.load("Config/explanation.toml")
mutants_config = toml.load("Config/mutants.toml")
pruning_config = toml.load("Config/pruning.toml")
pipeline_config = toml.load("Config/pipeline.toml")

In [2]:
def eval_config(config):
    for key in config.keys():
        if isinstance(config[key], dict):
            eval_config(config[key])
        else:
            try:
                if isfunction(eval(config[key])):
                    config[key] = eval(config[key])
            except:
                pass
    return config

adversarial_config = eval_config(adversarial_config)
explanation_config = eval_config(explanation_config)
mutants_config = eval_config(mutants_config)
pruning_config = eval_config(pruning_config)
pipeline_config = eval_config(pipeline_config)

In [3]:
pipeline_config

{'Config': {'model': 'custom', 'dataset': 'cifar10'},
 'Pruning': {'greg': {'input': 'train_dataloader',
   'output': 'greg_pruned_model_0_65'}},
 'Indicator': {'builtin': {'input': 'test_dataloader',
   'model': 'greg_pruned_model_0_65'}}}

In [4]:
model = importlib.import_module(f'Config.Model.{pipeline_config["Config"]["model"]}')
dataset = importlib.import_module(f'Config.Dataset.{pipeline_config["Config"]["dataset"]}')

In [5]:
TYPE = model.TYPE
if TYPE == "Image Classification":
    NUM_CLASSES = model.NUM_CLASSES
    INPUT_SIZE = model.INPUT_SIZE
elif TYPE == "Object Detection":
    INPUT_SIZE = model.INPUT_SIZE
elif TYPE == "Text Classification":
    NUM_CLASSES = model.NUM_CLASSES
ComposedModel = model.ComposedModel

In [6]:
model = ComposedModel().get_model()

In [7]:
train_dataset,test_dataset = dataset.load_dataset()

Files already downloaded and verified
Files already downloaded and verified


In [8]:
BATCH_SIZE = dataset.BATCH_SIZE
BATCH_SIZE

128

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [10]:
del pipeline_config['Config']

In [11]:
pipeline_config

{'Pruning': {'greg': {'input': 'train_dataloader',
   'output': 'greg_pruned_model_0_65'}},
 'Indicator': {'builtin': {'input': 'test_dataloader',
   'model': 'greg_pruned_model_0_65'}}}

In [12]:
def mutants_runner(module,f,function,cfg):
    input_ = globals()[cfg["input"]]
    function_config = mutants_config.get(f,{})
    function = function(**function_config)
    globals()[cfg["output"]] = DataLoader(input_.dataset,batch_size=BATCH_SIZE,shuffle=False,collate_fn=function)

In [13]:
def adversarial_runner(module,f,function,cfg):
    input_ = globals()[cfg["input"]]
    save = cfg["save"]
    function_config = adversarial_config.get(f,{})
    function = function(model,**function_config)
    if f == "advgan":
        function.train(train_dataloader)
    file_name = 0
    all_adversarial_labels = list()
    all_labels = list()
    correct_index = list()
    for x,y in input_:
        x = x.to(device)
        y = y.to(device)
        correct_idx = model(x).argmax(-1) == y
        correct_index.append(correct_idx.cpu().detach().numpy())
        adversarial_images = function(x,y)
        adverserial_labels = model(adversarial_images).argmax(-1).cpu().detach().numpy()
        all_labels.append(y.cpu().detach().numpy())
        if save:
            for adversarial_image in adversarial_images:
                adversarial_image = adversarial_image.cpu().detach().numpy().transpose(1,2,0)
                adversarial_image = Image.fromarray((adversarial_image * 255).astype(np.uint8))
                if not os.path.exists(f"Outputs/{experiment_name}/{module}/{f}"):
                    os.makedirs(f"Outputs/{experiment_name}/{module}/{f}")
                adversarial_image.save(f"Outputs/{experiment_name}/{module}/{f}/{file_name}.png")
                file_name += 1
        all_adversarial_labels.append(adverserial_labels)
    all_adversarial_labels = np.concatenate(all_adversarial_labels)
    all_labels = np.concatenate(all_labels)
    correct_index = np.concatenate(correct_index)
    success_rate = (1 - np.sum(all_adversarial_labels[correct_index] == all_labels[correct_index]) / np.sum(correct_index))
    print(f"{f} Success Rate: {success_rate * 100}%")
    if save:
        result = {
            "all_adversarial_labels":all_adversarial_labels,
            "all_labels":all_labels,
            "success_rate":success_rate
        }
        with open(f"Outputs/{experiment_name}/{module}/{f}/result.pkl","wb") as f:
            pickle.dump(result,f)
    

In [14]:
def explanation_runner(module,f,function,cfg):
    input_ = globals()[cfg["input"]]
    save = cfg["save"]
    function_config = explanation_config.get(f,{})
    for i,(x,y) in enumerate(input_):
        x = x.to(device)
        y = y.to(device)
        if f == "fast_ig" or f == "guided_ig":
            attribution = list()
            for x_,y_ in zip(x,y):
                x_ = x_.unsqueeze(0)
                y_ = y_.unsqueeze(0)
                attribution_ = function(model,x_,y_,**function_config)
                attribution.append(attribution_)
            attribution = np.concatenate(attribution,axis=0)
        else:
            attribution = function(model,x,y,**function_config)
        if save:
            if not os.path.exists(f"Outputs/{experiment_name}/{module}/{f}"):
                os.makedirs(f"Outputs/{experiment_name}/{module}/{f}")
            y = y.cpu().detach().numpy()
            x = x.cpu().detach().numpy()
            np.savez(f"Outputs/{experiment_name}/{module}/{f}/{i}.npz",x=x,y=y,attribution=attribution)
    if cfg['evaluate']:
        results_files = glob.glob(f"Outputs/{experiment_name}/{module}/{f}/*.npz")
        scores = {'del': [], 'ins': []}
        for file in results_files:
            results = np.load(file)
            attribution = results['attribution']
            x = torch.from_numpy(results['x']).to(device)
            deletion = CausalMetric(
                model, 'del', substrate_fn=torch.zeros_like, hw=np.prod(INPUT_SIZE[-2:]), num_classes=NUM_CLASSES)
            insertion = CausalMetric(
                model, 'ins', substrate_fn=torch.zeros_like, hw=np.prod(INPUT_SIZE[-2:]), num_classes=NUM_CLASSES)
            scores['del'].extend(deletion.evaluate(
                x, attribution, len(x)).tolist())
            scores['ins'].extend(insertion.evaluate(
                x, attribution, len(x)).tolist())
        scores['ins'] = np.array(scores['ins'])
        scores['del'] = np.array(scores['del'])
        with open(f'Outputs/{experiment_name}/{module}/{f}/scores.txt', 'w') as f:
            f.write("Insertion: " + str(scores['ins'].mean()) + "\n")
            f.write("Deletion: " + str(scores['del'].mean()) + "\n")
            print("Insertion: " + str(scores['ins'].mean()))
            print("Deletion: " + str(scores['del'].mean()))

In [15]:
def pruning_runner(module,f,function,cfg):
    input_ = globals()[cfg["input"]]
    function_config = pruning_config.get(f,{})
    save_path = f"Outputs/{experiment_name}/{module}/{f}/mask.pkl"
    if not os.path.exists(f"Outputs/{experiment_name}/{module}/{f}"):
        os.makedirs(f"Outputs/{experiment_name}/{module}/{f}")
    if not f == "greg":
        function = function(**function_config)
        globals()[cfg["output"]] = function(model,train_dataloader,save_path)
    else:
        function = function(**function_config)
        globals()[cfg["output"]] = function(model,train_dataloader,test_dataloader,save_path)

In [16]:
def distribution(module):
    return_dict = {
        "Mutants":mutants_runner,
        "Adversarial":adversarial_runner,
        "Pruning":pruning_runner,
        "Explanation":explanation_runner,
    }
    return return_dict[module]

In [17]:
last_module = None
last_f = None
for module,func in pipeline_config.items():
    for f,cfg in func.items():
        if module != "Indicator":
            md = importlib.import_module(module)
            function = getattr(md,f)
            print(f"Running {f} from {module}")
            runner = distribution(module)
            runner(module,f,function,cfg)
        else:
            md = importlib.import_module(f"{module}.{f}")
            metric_function = md.metric
            metrics, class_report = metric_function(globals()[cfg["model"]],globals()[cfg["input"]],num_classes=NUM_CLASSES)
            if not os.path.exists(f"Outputs/{experiment_name}/Indicator/{last_module}_{last_f}"):
                os.makedirs(f"Outputs/{experiment_name}/Indicator/{last_module}_{last_f}")
            metrics.to_csv(f'Outputs/{experiment_name}/Indicator/{last_module}_{last_f}/metrics.csv', index=False)
            class_report.to_csv(f'Outputs/{experiment_name}/Indicator/{last_module}_{last_f}/class_report.csv')
        last_f = f
    last_module = module
            
        

Running greg from Pruning
Register layer index and kernel shape:
[ 0]               model.conv1 -- kernel_shape: torch.Size([64, 3, 3, 3])
[ 1]      model.layer1.0.conv1 -- kernel_shape: torch.Size([64, 64, 1, 1])
[ 2]      model.layer1.0.conv2 -- kernel_shape: torch.Size([64, 64, 3, 3])
[ 3]      model.layer1.0.conv3 -- kernel_shape: torch.Size([256, 64, 1, 1])
[ 4] model.layer1.0.shortcut.0 -- kernel_shape: torch.Size([256, 64, 1, 1])
[ 5]      model.layer1.1.conv1 -- kernel_shape: torch.Size([64, 256, 1, 1])
[ 6]      model.layer1.1.conv2 -- kernel_shape: torch.Size([64, 64, 3, 3])
[ 7]      model.layer1.1.conv3 -- kernel_shape: torch.Size([256, 64, 1, 1])
[ 8]      model.layer1.2.conv1 -- kernel_shape: torch.Size([64, 256, 1, 1])
[ 9]      model.layer1.2.conv2 -- kernel_shape: torch.Size([64, 64, 3, 3])
[10]      model.layer1.2.conv3 -- kernel_shape: torch.Size([256, 64, 1, 1])
[11]      model.layer2.0.conv1 -- kernel_shape: torch.Size([128, 256, 1, 1])
[12]      model.layer2.0.con

  0%|          | 0/147 [00:00<?, ?it/s]

Acc1 = 95.4000 Acc5 = 99.7700 Iter = 0 (before update) [prune_state = update_reg]
==> [0] Just finished 'update_reg'. Iter = 0
Acc1 = 95.0400 Acc5 = 99.8400 Iter = 500 (before update) [prune_state = update_reg]
Acc1 = 95.2400 Acc5 = 99.8400 Iter = 1000 (before update) [prune_state = update_reg]
Acc1 = 95.2400 Acc5 = 99.8400 Iter = 1500 (before update) [prune_state = update_reg]
Acc1 = 95.3800 Acc5 = 99.8300 Iter = 2000 (before update) [prune_state = update_reg]
Acc1 = 95.2200 Acc5 = 99.8600 Iter = 2500 (before update) [prune_state = update_reg]
Acc1 = 95.2800 Acc5 = 99.8500 Iter = 3000 (before update) [prune_state = update_reg]
Acc1 = 95.4000 Acc5 = 99.8500 Iter = 3500 (before update) [prune_state = update_reg]
Acc1 = 95.4700 Acc5 = 99.8500 Iter = 4000 (before update) [prune_state = update_reg]
Acc1 = 95.4100 Acc5 = 99.8700 Iter = 4500 (before update) [prune_state = update_reg]
Acc1 = 95.4700 Acc5 = 99.8600 Iter = 5000 (before update) [prune_state = update_reg]
Acc1 = 95.5500 Acc5 = 99

KeyboardInterrupt: 

In [2]:
from Config.Model.vgg16_imagenet import ComposedModel

In [3]:
ComposedModel().get_model().num_layers

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

16